# ATP Synthase Model Attempt 2
7.8.2020<br>
In this notebook, we will try to implement a control system for H+ concentration.

In [1]:
# Import cell (refined)
import numpy as np
import pandas as pd
import scipy.special as sp
import scipy.optimize
from scipy.optimize import curve_fit
import scipy.integrate
from scipy.integrate import odeint

import biocircuits

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row

bokeh.io.output_notebook()



Loading BokehJS ...

In [89]:
def rhs(var, t, k1, k2, k3):
    
    atp,adp,pi,h,h20,outside_H = var
    
    
    # PH CONTROL PORTION WHEN PH IS GREATER THAN 8, ATP SYNTHASE RUNS BACKWARDS
    

    datp_dt = k1 * adp * pi * h**3.3 - k2 * atp
    
    dadp_dt = -k1 * adp * pi * h**3.3 + k2 * atp
    
    dpi_dt = dadp_dt
    
    dh_dt = k3 * outside_H - k1 * adp * pi * h**4
    
    dh20_dt = k1 * adp * pi * h**4

    doutsideH_dt = H_control(var, t, k3)
    
    return np.array([datp_dt, dadp_dt, dpi_dt, dh_dt, dh20_dt, doutsideH_dt])
    


In [106]:
def H_control(var, t, k3):
    
    atp,adp,pi,h,h20,outside_H = var
    
    if t < 2:
        N = 0

    elif t > 2 and t <= 2.2:
        N = 30
    
    elif t > 20 and t <= 20.2:
        N = 30
        
    else:
        N = -k3 * outside_H
    
    return N

Some rate justifications: 
- ATP synthesis rate is > 100 ATP/sec (bionum ID 111975) or 6,000 ATP/min or 360,000 ATP/hr <br>
- ATP Use rate in Tx/Tl? - we normally assume there is none by hour 20<br>
- Each ATP requires the transport or 3.33 protons, we round to 4. <br>
- what is k3: outside H to H rate: if 3.33 h+ atp/second and 100 atp/s then 400 h+/sec or 24000 h+ / min or 1,440,000 h+ /hr

In [107]:
t_vals = np.linspace(0,50,201)
k1 = 100 * 60 * 60 # per sec to hr
k2 = 100 * 60 * 60
k3 = 333 * 60 * 60 # per sec to hr

k1 = 0.1 # per sec to hr
k2 = 0.1
k3 = 0.1 # per sec to hr

x_args = (k1, k2, k3)
y0 = np.array([30, 0, 0, 0, 0, 0])

x = odeint(rhs, y0, t_vals, args = x_args)

colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']



p = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",
    title = 'All Species'
        )
p1 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'Outside H+'
        )

spec_list = ['atp', 'adp', 'pi', 'h+', 'h20', 'outside h+']

for i,spec in enumerate(spec_list):
    if spec_list[i] == 'h20':
        i+=1
    else:
        p.line(t_vals, x[:,i], line_width = 2, color = colors[i], legend_label = spec)
        

p1.line(t_vals, x[:,5], line_width = 2, color= colors[5] )
        
        
p.legend.click_policy = 'hide'


bokeh.io.show(row(p, p1))

In [9]:
odeint?

Signature:
odeint(
    func,
    y0,
    t,
    args=(),
    Dfun=None,
    col_deriv=0,
    full_output=0,
    ml=None,
    mu=None,
    rtol=None,
    atol=None,
    tcrit=None,
    h0=0.0,
    hmax=0.0,
    hmin=0.0,
    ixpr=0,
    mxstep=0,
    mxhnil=0,
    mxordn=12,
    mxords=5,
    printmessg=0,
    tfirst=False,
)
Docstring:
Integrate a system of ordinary differential equations.

.. note:: For new code, use `scipy.integrate.solve_ivp` to solve a
          differential equation.

Solve a system of ordinary differential equations using lsoda from the
FORTRAN library odepack.

Solves the initial value problem for stiff or non-stiff systems
of first order ode-s::

    dy/dt = func(y, t, ...)  [or func(t, y, ...)]

where y can be a vector.

.. note:: By default, the required order of the first two arguments of
          `func` are in the opposite order of the arguments in the system
          definition function used by the `scipy.integrate.ode` class and
          the function `